# ASR Commands Model Evaluation

This notebook loads the archived ASR commands model and evaluates it on the test set. Metrics and visualizations are saved to the evaluation directory.

In [ ]:
# Setup: imports and paths
import sys
from pathlib import Path
import json
import numpy as np
import matplotlib.pyplot as plt

sys.path.insert(0, str(Path('../../utils').resolve()))
from utils.paths import CACHE_PATH, ROOT_PATH

EVAL_DIR = ROOT_PATH / 'evaluation/asr_commands'
ARCHIVE_PATH = ROOT_PATH / 'models/asr_commands.zip'
CACHE_DIR = CACHE_PATH / 'models/asr_commands'
SPLITS_PATH = ROOT_PATH / 'outputs/asr_commands/preprocessing/splits.json'

sys.path.insert(0, str(EVAL_DIR))
from eval_utils import compute_metrics
from model_utils import load_model_and_extractor, load_test_dataset, preprocess_test_dataset
from unpack_model import unpack_model

In [ ]:
# Unpack model to cache dir (if needed)
unpack_model(ARCHIVE_PATH, CACHE_DIR)
# Load test split and labels
test_ds, labels = load_test_dataset(SPLITS_PATH)

In [ ]:
# Load model and feature extractor
model, feature_extractor, device = load_model_and_extractor(CACHE_DIR)

In [ ]:
# Preprocess test set
test_ds_proc = preprocess_test_dataset(test_ds, feature_extractor)

In [ ]:
# Run inference
all_preds, all_labels = [], []
with torch.no_grad():
    for ex in test_ds_proc:
        inp = torch.tensor(ex['input_values']).unsqueeze(0).to(device)
        logits = model(inp).logits.cpu().numpy()[0]
        pred = np.argmax(logits)
        all_preds.append(pred)
        all_labels.append(ex['label'])
all_preds, all_labels = np.array(all_preds), np.array(all_labels)

In [ ]:
# Compute and save metrics
metrics = compute_metrics(all_labels, all_preds, labels)
with open(EVAL_DIR / 'metrics.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2)
metrics

In [ ]:
# Visualize confusion matrix
cm = np.array(metrics['confusion_matrix'])
fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yticklabels(labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.colorbar(im)
plt.tight_layout()
plt.savefig(EVAL_DIR / 'confusion_matrix.png')
plt.show()

## Results
- Metrics are saved to `metrics.json` in the evaluation directory.
- Confusion matrix is saved as `confusion_matrix.png`.
- See the classification report in the metrics output above.